# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV 

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Hobbies/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [6]:
sales_erratic

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,...,0,0,1,37,3,4,6,3,2,1
14,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,4,0,0,5,...,1,4,2,2,1,0,0,2,5,4
15,HOBBIES_1_016_CA_1_validation,HOBBIES_1_016,HOBBIES_1,HOBBIES,CA_1,CA,5,1,3,0,...,5,6,15,9,4,4,0,15,1,6
29,HOBBIES_1_030_CA_1_validation,HOBBIES_1_030,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,12,0,8,7,7,19,3,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9486,HOBBIES_1_348_CA_4_validation,HOBBIES_1_348,HOBBIES_1,HOBBIES,CA_4,CA,1,4,12,12,...,8,5,3,14,8,0,14,10,13,6
9508,HOBBIES_1_370_CA_4_validation,HOBBIES_1_370,HOBBIES_1,HOBBIES,CA_4,CA,9,13,2,8,...,3,5,0,3,7,10,1,3,8,7
9509,HOBBIES_1_371_CA_4_validation,HOBBIES_1_371,HOBBIES_1,HOBBIES,CA_4,CA,2,11,33,7,...,5,3,16,16,6,9,14,0,0,9
9519,HOBBIES_1_381_CA_4_validation,HOBBIES_1_381,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,8,6,0,0,0,0,6,1,4,5


# User-defined functions to calculate Metrics

In [7]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [8]:
def data_cleaning(sales_pattern):

    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()

    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')

    df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
    df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
    df_pattern.reset_index(inplace=True)

    calendar['Date'] = pd.to_datetime(calendar['date'])

    dataframe = pd.merge(df_pattern,
                         calendar[['wday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'Date']],
                         on = 'Date')

    dataframe['weekend'] = np.where(((dataframe['wday'] == 1) | (dataframe['wday'] == 2)), 1, 0)
    dataframe.fillna('No event', inplace=True)
    dataframe['year'] = dataframe['year'].apply(str)

    dataframe['quarter'] = dataframe['Date'].dt.quarter
    dataframe['quarter'] = dataframe['quarter'].apply(str)

    dataframe = pd.get_dummies(dataframe)
    
    return dataframe

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [9]:
data_pattern = data_cleaning(sales_erratic)

In [10]:
data_pattern

,Date,HOBBIES_1_004_CA_1,HOBBIES_1_008_CA_1,HOBBIES_1_008_CA_3,HOBBIES_1_014_CA_3,HOBBIES_1_015_CA_1,HOBBIES_1_015_CA_3,HOBBIES_1_015_CA_4,HOBBIES_1_016_CA_1,HOBBIES_1_016_CA_3,...,event_name_1_VeteransDay,event_type_1_Cultural,event_type_1_National,event_type_1_No event,event_type_1_Religious,event_type_1_Sporting,quarter_1,quarter_2,quarter_3,quarter_4
0,2011-01-29,0.0,12.0,11.0,0.0,4.0,22.0,2.0,5.0,21.0,...,0,0,0,1,0,0,1,0,0,0
1,2011-01-30,0.0,0.0,15.0,0.0,0.0,0.0,0.0,7.0,22.0,...,0,0,0,1,0,0,1,0,0,0
2,2011-01-31,3.0,0.0,36.0,0.0,0.0,38.0,6.0,73.0,8.0,...,0,0,0,1,0,0,1,0,0,0
3,2011-02-01,1.0,5.0,6.0,1.0,1.0,5.0,0.0,4.0,0.0,...,0,0,0,1,0,0,1,0,0,0
4,2011-02-02,0.0,10.0,8.0,0.0,1.0,8.0,6.0,1.0,7.0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,1.0,6.0,0.0,3.0,10.0,12.0,3.0,8.0,10.0,...,0,0,0,1,0,0,0,1,0,0
1909,2016-04-21,2.0,21.0,0.0,1.0,19.0,20.0,5.0,4.0,4.0,...,0,0,0,1,0,0,0,1,0,0
1910,2016-04-22,2.0,0.0,4.0,0.0,12.0,3.0,8.0,0.0,3.0,...,0,0,0,1,0,0,0,1,0,0
1911,2016-04-23,1.0,13.0,6.0,1.0,2.0,13.0,3.0,3.0,8.0,...,0,0,0,1,0,0,0,1,0,0


# Random Forest Hyperparameters to find optimum parameters

In [11]:
not_id_cols = [
 'Date',
 'wday',
 'month',
 'snap_CA',
 'weekend',
 'year_2011',
 'year_2012',
 'year_2013',
 'year_2014',
 'year_2015',
 'year_2016',
 'event_name_1_Chanukah End',
 'event_name_1_Christmas',
 'event_name_1_Cinco De Mayo',
 'event_name_1_ColumbusDay',
 'event_name_1_Easter',
 'event_name_1_Eid al-Fitr',
 'event_name_1_EidAlAdha',
 "event_name_1_Father's day",
 'event_name_1_Halloween',
 'event_name_1_IndependenceDay',
 'event_name_1_LaborDay',
 'event_name_1_LentStart',
 'event_name_1_LentWeek2',
 'event_name_1_MartinLutherKingDay',
 'event_name_1_MemorialDay',
 "event_name_1_Mother's day",
 'event_name_1_NBAFinalsEnd',
 'event_name_1_NBAFinalsStart',
 'event_name_1_NewYear',
 'event_name_1_No event',
 'event_name_1_OrthodoxChristmas',
 'event_name_1_OrthodoxEaster',
 'event_name_1_Pesach End',
 'event_name_1_PresidentsDay',
 'event_name_1_Purim End',
 'event_name_1_Ramadan starts',
 'event_name_1_StPatricksDay',
 'event_name_1_SuperBowl',
 'event_name_1_Thanksgiving',
 'event_name_1_ValentinesDay',
 'event_name_1_VeteransDay',
 'event_type_1_Cultural',
 'event_type_1_National',
 'event_type_1_No event',
 'event_type_1_Religious',
 'event_type_1_Sporting',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4'
]

In [12]:
end_train_date = '2016-04-10'

In [13]:
lags = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28]
moving_averages = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
stds = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
maximums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
minimums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
totals = [2, 3, 4, 5, 6, 7, 14, 21, 28]
num_days_zeros = [2, 3, 4, 5, 6, 7, 14, 21, 28] 

In [14]:
correlation_threshold = 0.8

In [15]:
n_estimators = [30, 50, 100, 150]
max_depth = list(range(4,9)) + [None]
min_samples_leaf = range(4,11)
max_features = range(20,30)
bootstrap = [True] #We force bootstrap
max_samples = [0.6, 0.7, 0.8, 0.9]

In [16]:
# User-defined funtion to find optimum parameters on training data
def Random_Forest_Hyperparameters_Tuning(list_pattern, not_id_cols, lags, moving_averages, stds,
                          maximums, minimums, totals, num_days_zeros, correlation_threshold,
                          end_train_date, n_estimators, max_depth, min_samples_leaf,
                          max_features, bootstrap, max_samples):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_hyper_params = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))

        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        param_dist = {'n_estimators': n_estimators,
                      'max_depth': max_depth,
                      'min_samples_leaf': min_samples_leaf,
                      'max_features': max_features,
                      'bootstrap': bootstrap,
                      'max_samples': max_samples}

        forest = RandomForestRegressor(n_jobs=1)

        forest_cv = RandomizedSearchCV(forest, param_dist, cv=5, n_jobs=-1, verbose=2, n_iter=300, scoring='neg_mean_absolute_error')
        forest_cv.fit(X_train,y_train)
            
        best_params_temp = forest_cv.best_params_
        
        df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
        df_output_temp['Product'] = product
        
        df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)
        
    return df_output_hyper_params

**Sample 30 products due to limited computational power**

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [ ]:
list_pattern_sample = sample(list_erratic, 30)

In [ ]:
list_pattern_sample

['FOODS_3_428_CA_3_validation',
 'FOODS_3_644_CA_1_validation',
 'FOODS_3_455_CA_2_validation',
 'FOODS_3_399_CA_3_validation',
 'FOODS_3_267_CA_1_validation',
 'FOODS_3_499_CA_2_validation',
 'FOODS_3_476_CA_1_validation',
 'FOODS_3_801_CA_2_validation',
 'FOODS_1_046_CA_4_validation',
 'FOODS_1_018_CA_3_validation',
 'FOODS_3_185_CA_1_validation',
 'FOODS_3_295_CA_4_validation',
 'FOODS_3_583_CA_3_validation',
 'FOODS_3_481_CA_3_validation',
 'FOODS_2_321_CA_3_validation',
 'FOODS_3_363_CA_2_validation',
 'FOODS_3_809_CA_3_validation',
 'FOODS_3_367_CA_2_validation',
 'FOODS_3_672_CA_3_validation',
 'FOODS_2_293_CA_1_validation',
 'FOODS_3_310_CA_3_validation',
 'FOODS_3_801_CA_3_validation',
 'FOODS_3_282_CA_2_validation',
 'FOODS_3_458_CA_2_validation',
 'FOODS_1_200_CA_3_validation',
 'FOODS_3_596_CA_3_validation',
 'FOODS_3_339_CA_1_validation',
 'FOODS_3_607_CA_2_validation',
 'FOODS_3_346_CA_1_validation',
 'FOODS_1_021_CA_1_validation']

**Start Hyperparameters tuning process**

In [ ]:
start = timer()

df_hyper_params = Random_Forest_Hyperparameters_Tuning(list_pattern_sample, not_id_cols, lags, moving_averages, stds,
                                                       maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                                       end_train_date, n_estimators, max_depth, min_samples_leaf,
                                                       max_features, bootstrap, max_samples)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_3_428_CA_3
Progressing: 0.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: FOODS_3_644_CA_1
Progressing: 3.33 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: FOODS_3_455_CA_2
Progressing: 6.67 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: FOODS_3_399_CA_3
Progressing: 10.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: FOODS_3_267_CA_1
Progressing: 13.33 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: FOODS_3_499_CA_2
Progressing: 16.67 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: FOODS_3_476_CA_1
Progressing: 20.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: FOODS_3_801_CA_2
Progressing: 23.33 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: FOODS_1_046_CA_4
Pr

In [ ]:
df_hyper_params

,n_estimators,min_samples_leaf,max_samples,max_features,max_depth,bootstrap,Product
0,30,8,0.7,28,6,True,FOODS_3_428_CA_3
1,30,4,0.8,21,8,True,FOODS_3_644_CA_1
2,30,6,0.6,23,8,True,FOODS_3_455_CA_2
3,50,6,0.6,26,8,True,FOODS_3_399_CA_3
4,30,6,0.6,23,8,True,FOODS_3_267_CA_1
5,100,6,0.9,28,8,True,FOODS_3_499_CA_2
6,100,9,0.6,23,None,True,FOODS_3_476_CA_1
7,30,10,0.6,23,6,True,FOODS_3_801_CA_2
8,30,10,0.8,24,7,True,FOODS_1_046_CA_4
9,30,6,0.6,26,8,True,FOODS_1_018_CA_3


In [ ]:
df_hyper_params.to_csv('Random_Forest_Hyperparams_Erratic.csv')

# Decide optimum set of parameters for Random Forest algorithm

In [ ]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_hyper_params.groupby(['n_estimators', 'min_samples_leaf', 'max_samples', 
                         'max_features', 'max_depth', 'bootstrap'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
n_estimators min_samples_leaf max_samples max_features max_depth bootstrap         
30           6                0.6         23           8         True             2
             4                0.7         20           7         True             1
50           7                0.6         25           7         True             1
150          5                0.6         23           8         True             1
             4                0.8         29           7         True             1
                              0.7         25           7         True             1
100          10               0.7         27           4         True             1
             7                0.8         20           7         True             1
             6                0.9         28           8         True             1
             4                0.9         27           8         True             1
                                                       7         True             1
50           10               0.8         24           5         True             1
             9                0.6         21           6         True             1
             6                0.6         26           8         True             1
30           4                0.8         21           8         True             1
50           5                0.7         20           7         True             1
             4                0.6         25           5         True             1
30           10               0.8         24           7         True             1
                              0.6         23           6         True             1
             9                0.9         29           8         True             1
             8                0.9         27           5         True             1
                              0.7         28           6         True             1
             6                0.8         23           4         True             1
                              0.6         26           8         True             1
150          6                0.8         28           8         True             1

In [ ]:
# See what value of parameter "n_estimators" occurs the most frequently
df_hyper_params.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
30,11
50,8
100,7
150,4


In [ ]:
# See what value of parameter "min_samples_leaf" occurs the most frequently
df_hyper_params.groupby(['min_samples_leaf'])[['Product']].count()

,Product
min_samples_leaf,
4,9
5,2
6,7
7,2
8,2
9,4
10,4


In [ ]:
# See what value of parameter "max_samples" occurs the most frequently
df_hyper_params.groupby(['max_samples'])[['Product']].count()

,Product
max_samples,
0.6,11
0.7,5
0.8,7
0.9,7


In [ ]:
# See what value of parameter "max_features" occurs the most frequently
df_hyper_params.groupby(['max_features'])[['Product']].count()

,Product
max_features,
20,3
21,2
22,1
23,6
24,2
25,4
26,2
27,5
28,3


In [ ]:
# See what value of parameter "max_depth" occurs the most frequently
df_hyper_params.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
4,2
5,3
6,3
7,8
8,10


# Implement Random Forest on Test Set with optimum set of parameters

**Specify optimum set of parameters**

In [17]:
n_estimators_chosen = 30
min_samples_leaf_chosen = 6
max_samples_choose = 0.6
max_features_choose = 23
max_depth_choose = 8
bootstrap = [True]

In [18]:
# User-defined function to run Random Forest on Test Data

def Random_Forest_Test_Data(list_pattern, not_id_cols, lags, moving_averages, stds,
                          maximums, minimums, totals, num_days_zeros, correlation_threshold,
                          end_train_date, n_estimators_chosen, min_samples_leaf_chosen,
                          max_samples_choose, max_features_choose, max_depth_choose, bootstrap):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_rf_pattern = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running Product: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))

        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]
        test_data = dataframe_product[dataframe_product.index > end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        X_test = test_data.drop(product, axis=1)
        y_test = test_data[product]

        forest = RandomForestRegressor(n_jobs=1,
                                      n_estimators = n_estimators_chosen,
                                      min_samples_leaf = min_samples_leaf_chosen,
                                      max_samples = max_samples_choose,
                                      max_features = max_features_choose,
                                      max_depth = max_depth_choose,
                                      bootstrap = bootstrap,
                                      random_state = 0)

        forest.fit(X_train,y_train)
        y_pred = forest.predict(X_test)
        df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                       'Forecast': y_pred, 
                                       'Product': [product for num_rows in range(len(y_pred))]
                                       })
        
        df_output_rf_pattern = df_output_rf_pattern.append(df_output_temp, ignore_index=False)
        
    return df_output_rf_pattern

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [19]:
# Start running Random Forest on Test Set

start = timer()

df_forecast_values = Random_Forest_Test_Data(list_erratic, not_id_cols, lags, moving_averages, stds,
                                             maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                             end_train_date, n_estimators_chosen, min_samples_leaf_chosen,
                                             max_samples_choose, max_features_choose, max_depth_choose, bootstrap)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: HOBBIES_1_004_CA_1
Progressing: 0.0 %
Currently Running Product: HOBBIES_1_008_CA_1
Progressing: 1.49 %
Currently Running Product: HOBBIES_1_008_CA_3
Progressing: 2.99 %
Currently Running Product: HOBBIES_1_014_CA_3
Progressing: 4.48 %
Currently Running Product: HOBBIES_1_015_CA_1
Progressing: 5.97 %
Currently Running Product: HOBBIES_1_015_CA_3
Progressing: 7.46 %
Currently Running Product: HOBBIES_1_015_CA_4
Progressing: 8.96 %
Currently Running Product: HOBBIES_1_016_CA_1
Progressing: 10.45 %
Currently Running Product: HOBBIES_1_016_CA_3
Progressing: 11.94 %
Currently Running Product: HOBBIES_1_019_CA_3
Progressing: 13.43 %
Currently Running Product: HOBBIES_1_019_CA_4
Progressing: 14.93 %
Currently Running Product: HOBBIES_1_030_CA_1
Progressing: 16.42 %
Currently Running Product: HOBBIES_1_043_CA_3
Progressing: 17.91 %
Currently Running Product: HOBBIES_1_048_CA_1
Progressing: 19.4 %
Currently Running Product: HOBBIES_1_048_CA_3
Progressing: 20.9 %
Curre

In [20]:
df_forecast_values

,Actual Data,Forecast,Product
Date,,,
2016-04-11,0.0,1.773435,HOBBIES_1_004_CA_1
2016-04-12,3.0,1.888851,HOBBIES_1_004_CA_1
2016-04-13,5.0,2.485358,HOBBIES_1_004_CA_1
2016-04-14,1.0,1.526874,HOBBIES_1_004_CA_1
2016-04-15,1.0,1.316235,HOBBIES_1_004_CA_1
...,...,...,...
2016-04-20,6.0,6.998174,HOBBIES_1_404_CA_4
2016-04-21,1.0,7.572080,HOBBIES_1_404_CA_4
2016-04-22,3.0,9.004848,HOBBIES_1_404_CA_4


In [21]:
df_forecast_values.to_csv('Random Forest_Erratic_Forecast_Values.csv')

In [22]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [23]:
# Start calculating metrics 
start = timer()

df_forecast_metrics = df_forecast_values.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0037470807333344664 minutes


In [24]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_004_CA_1,0.713868,49.4216,55.1270,46.6282,0.988432,1.239537
1,HOBBIES_1_008_CA_1,0.618764,92.9681,109.1188,250.3577,8.234315,9.926128
2,HOBBIES_1_008_CA_3,0.748855,90.9411,93.8479,165.6325,6.106047,6.745593
3,HOBBIES_1_014_CA_3,0.702839,103.2141,101.8407,99.7935,2.432905,2.915192
4,HOBBIES_1_015_CA_1,0.761077,61.8734,65.0133,114.2277,4.507919,5.198197
...,...,...,...,...,...,...,...
62,HOBBIES_1_381_CA_4,0.758089,121.5534,135.7246,80.3941,3.907073,4.428898
63,HOBBIES_1_387_CA_3,0.545420,64.5222,72.4899,86.6993,6.083525,8.576657
64,HOBBIES_1_404_CA_1,0.637226,78.9978,82.1318,123.7029,4.852723,6.561693
65,HOBBIES_1_404_CA_2,0.661665,79.0015,91.6844,85.1823,5.191528,8.419589


In [25]:
df_forecast_values.to_csv('Random Forest_Erratic_Forecast_Metrics.csv')